# Human retina sample scRNA-seq data analysis workflow
# Running each of these cells will go through and implement the individual stages of the scRNA-seq data analysis workflow

### Importing the time module to record the running time of individual cells in the Jupyter notebook 

In [ ]:
import time
start_time = time.time()

### Installing all the packages required for the various stages of analysis

In [ ]:
%%time
# These packages are pre-installed on Google Colab, but are included here to simplify running this notebook locally
!pip install matplotlib
!pip install scikit-learn
!pip install numpy
!pip install scipy
!pip install leidenalg
!pip install scanpy

### Installing the kb-python package for the processing scRNA-seq reads

In [ ]:
%%time
!pip install kb-python 

### Downloading the FASTQ files from the ArrayExpress https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-7316/samples/
### using the wget command

In [ ]:
%%time
!wget --continue ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR842/008/SRR8426358/SRR8426358_1.fastq.gz
!wget --continue ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR842/008/SRR8426358/SRR8426358_2.fastq.gz

### Downloading a human referecne transcriptome index and a transcript to gene map using the kb ref command

In [ ]:
!kb ref -d human -i index.idx -g t2g.txt -f1 transcriptome.fasta

### Using the kb command to process the scRNA-seq data downloaded using the wget command
### This command pseudoaligns and counts reads to produce a cells x genes unfiltered count matrix 
###  Unfiltered count matrix is generated in adata.h5ad file format

In [ ]:
%%time
!kb count --h5ad -i index.idx -g t2g.txt -x 10xv2 -o output --filter bustools -t 2 \
ae_exp_raw_r1_R1.fastq.gz \
ae_exp_raw_r1_R2.fastq.gz

### Setting up QC by importing the necessary modules and packagaes

In [ ]:
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as sc
from scipy import stats

from collections import OrderedDict
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
from matplotlib.lines import Line2D

def nd(arr):
    return np.asarray(arr).reshape(-1)
def yex(ax):
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

def trim_axs(axs, N):
    """little helper to massage the axs list to have correct length..."""
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

### Loading in the unfiltered count matrix 

In [ ]:
adata = anndata.read_h5ad("output/counts_unfiltered/adata.h5ad")
adata.var["gene_id"] = adata.var.index.values

t2g = pd.read_csv("t2g.txt", header=None, names=["tid", "gene_id", "gene_name"], sep="\t")
t2g.index = t2g.gene_id
t2g = t2g.loc[~t2g.index.duplicated(keep='first')]

adata.var["gene_name"] = adata.var.gene_id.map(t2g["gene_name"])
adata.var.index = adata.var["gene_name"]

### Reading the shape of the unfiltered count matrix to see the number of cells and genes in the matrix 

In [ ]:
adata

### Setting QC and filtering parameters

In [ ]:
num_TSNE = 2
state = 42
metric = "euclidean"
n_neighbors = 30
num_PCA = 50
num_NCA = 10

# Filtering parameters

cell_threshold = 100 
gene_threshold = 3 
mito_criteria = 30 

n_top_genes = 5000 # Top 5000 most variable genes selected for PCA

n_bins = 20

flavor="seurat"

In [ ]:
adata.obs["cell_counts"] = adata.X.sum(axis=1)
adata.var["gene_counts"] = nd(adata.X.sum(axis=0))

adata.obs["n_genes"] = nd((adata.X>0).sum(axis=1))
adata.var["n_cells"] = nd((adata.X>0).sum(axis=0))

mito_genes = adata.var_names.str.startswith('mt-')
adata.obs["percent_mito"] = adata[:,mito_genes].X.sum(axis=1)/adata.X.sum(axis=1)*100

### Generating a knee plot

In [ ]:
#@title Threshold cells according to knee plot { run: "auto", vertical-output: true }
expected_num_cells = 3000#@param {type:"integer"}
knee = np.sort(nd(adata.X.sum(axis=1)))[::-1]

fig, ax = plt.subplots(figsize=(5, 5))

x = knee
y = range(len(knee))

ax.loglog(x, y, linewidth=5, color="g")

ax.axvline(x=knee[expected_num_cells], linewidth=3, color="k")
ax.axhline(y=expected_num_cells, linewidth=3, color="k")

ax.set_xlabel("UMI Counts")
ax.set_ylabel("Set of Barcodes")

plt.show()

### Generating a percentage (%) mitchondrial read plot

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

x = nd(adata.obs["cell_counts"])
y = nd(adata.obs["percent_mito"])

ax.scatter(x, y, color="green", alpha=0.25)

ax.axhline(y=mito_criteria, linestyle="--", color="k")

ax.set_xlabel("UMI Counts")
ax.set_ylabel("Percent mito")

plt.show()

### Applying filtering parameters to filter out low quality cells
### Printing the shape of unfiltered and filtered count matrix to see how many cells and genes have been filtered out

In [ ]:
adata.obs["pass_count_filter"] = adata.obs["cell_counts"] > cell_threshold
adata.obs["pass_mito_filter"] = adata.obs.percent_mito < mito_criteria
adata.var["pass_gene_filter"] = adata.var["n_cells"] > gene_threshold

In [ ]:
cell_mask = np.logical_and(adata.obs["pass_count_filter"].values, adata.obs["pass_mito_filter"].values)
gene_mask = adata.var["pass_gene_filter"].values

In [ ]:
print("Current Shape: {:,} cells x {:,} genes".format(adata.shape[0], adata.shape[1]))
print("    New shape: {:,} cells x {:,} genes".format(cell_mask.sum(), gene_mask.sum()))

## From now on the filtered count matrix will be used for analysis 

In [ ]:
data = adata[cell_mask, gene_mask]

### Generating a library saturation plot

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

x = nd(data.X.sum(axis=1))
y = nd(np.sum(data.X>0, axis=1))

ax.scatter(x, y, color="green", alpha=0.25)

ax.set_xlabel("UMI Counts")
ax.set_ylabel("Genes Detected")
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlim(1)
ax.set_ylim(1)

plt.show()

### Generating a data density plot

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

#histogram definition
bins = [1500, 1500] # number of bins

# histogram the data
hh, locx, locy = np.histogram2d(x, y, bins=bins)

# Sort the points by density, so that the densest points are plotted last
z = np.array([hh[np.argmax(a<=locx[1:]),np.argmax(b<=locy[1:])] for a,b in zip(x,y)])
idx = z.argsort()
x2, y2, z2 = x[idx], y[idx], z[idx]

s = ax.scatter(x2, y2, c=z2, cmap='Greens')  
fig.colorbar(s, ax=ax)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("UMI Counts")
ax.set_ylabel("Genes Detected")

ax.set_xlim(1, 10**5)
ax.set_ylim(1, 10**4)

plt.show()

### Reading in the shape of the filtered count matrix 

In [ ]:
data

### Using Scanpy to generate a plot of the top 20 genes with the highest fraction of counts across all cells

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
sc.pl.highest_expr_genes(data, n_top=20, ax = ax)

### Setting up the violin plot 
### The violin plot displays the distribution in the number genes detected, UMI counts and percentage (%) reads mapping to mitochondrial genes

In [ ]:
def vplot(y, ax):
    parts = ax.violinplot(
        y, showmeans=False, showmedians=False,
        showextrema=False)

    mean = y.mean()
    ax.scatter(1, mean, zorder=10, color="white")
    
    x = np.random.normal(1, 0.04, size=len(y))
    ax.scatter(x, y, color="k", s=1)
    
    for pc in parts['bodies']:
        pc.set_facecolor('#D43F3A')
        pc.set_edgecolor('black')
        pc.set_alpha(1)
    
    ax.set_xticks([1])
    ax.set_xticklabels([""])
    return ax

### Generating the violin plot

In [ ]:
fig, ax = plt.subplots(figsize=(5*3,5), ncols=3)

x1 = data.obs["n_genes"]
x2 = nd(data.X.sum(axis=1))
x3 = data.obs["percent_mito"]

vplot(x1, ax[0])
vplot(x2, ax[1])
vplot(x3, ax[2])

ax[0].set_ylabel("Genes detected")
ax[1].set_ylabel("UMI counts")
ax[2].set_ylabel("Percent Mito")

plt.tight_layout()
plt.show()

### Processing the matrix prior to clustering, data visualisation and differential expression

### Normalizing counts by converting them into CPM (counts per million) units and then log(CPM+1)

In [ ]:
data.layers["raw"] = data.X

In [ ]:
scale_num = 1000000
data.layers["norm"] = normalize(data.X, norm="l1", axis=1)*scale_num

In [ ]:
data.layers["log1p"] = np.log1p(data.layers["norm"])
data.uns = OrderedDict([("log1p", {"base":None})])

In [ ]:
data.X = data.layers["log1p"]

### Using Scanpy to detect highly variable genes in the normalized data

In [ ]:
sc.pp.highly_variable_genes(data, n_top_genes=n_top_genes, flavor=flavor, n_bins=n_bins)
hvg_mask = data.var.highly_variable.values

### Making a dense matrix as the scaling operation cannot be performed on a sparse matrix 

In [ ]:
%%time
mat = data.layers["log1p"].todense()
data.layers["scale"] = scale(mat, axis=0, with_mean=True, with_std=True, copy=True)
data.X = data.layers["scale"]

del mat

### Performing PCA (Principal components analysis) on the highly variable genes

In [ ]:
%%time
X = data.X[:,hvg_mask]

tsvd = TruncatedSVD(n_components=num_PCA)
data.obsm["X_pca"] = tsvd.fit_transform(X)

### Using Scanpy to compute a neighbourhood graph. Then using Scanpy to cluster the cells, using the Leiden algorithm  

In [ ]:
sc.pp.neighbors(data, n_neighbors=n_neighbors, n_pcs=num_PCA, random_state=state)

In [ ]:
sc.tl.leiden(data, random_state=state)

### Performing dimensionality reduction using t-SNE (t-Distributed Stochastic Neighbor Embedding)

In [ ]:
X = data.obsm["X_pca"]

tsne = TSNE(n_components=num_TSNE, metric=metric, random_state=state)
data.obsm["X_pca_tsne"] = tsne.fit_transform(X)

### Performing NCA (Neighbourhood components analysis)

In [ ]:
X = data.X
y = data.obs.leiden.values 

nca = NeighborhoodComponentsAnalysis(n_components=num_NCA,random_state=state)
data.obsm["X_nca"] = nca.fit_transform(X, y)

### Generating a t-SNE of the NCA projection

In [ ]:
X = data.obsm["X_nca"]
tsne = TSNE(n_components=num_TSNE, metric=metric, random_state=state)
data.obsm["X_nca_tsne"] = tsne.fit_transform(X)

### Visualisation of PCA followed by t-SNE

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))

x = data.obsm["X_pca_tsne"][:,0]
y = data.obsm["X_pca_tsne"][:,1]
c = data.obs["leiden"].astype(int)

ax.scatter(x, y, c = c, cmap='tab20')

ax.set_axis_off()

plt.tight_layout()
plt.show()

### Visualisation of NCA followed by t-SNE

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))

x = data.obsm["X_nca_tsne"][:,0]
y = data.obsm["X_nca_tsne"][:,1]
c = data.obs["leiden"].astype(int)

ax.scatter(x, y, c = c, cmap='tab20')

ax.set_axis_off()

plt.tight_layout()
plt.show()

## Finding marker genes using different statistical methods

### Using the t-test to rank the top 25 genes in each cluster

### Genes in each cluster are ranked accroding to how different they are relative to all other clusters

In [ ]:
sc.tl.rank_genes_groups(data, 'leiden', method='t-test', corr_method="bonferroni")
sc.pl.rank_genes_groups(data, n_genes=25, sharey=False)

### Using the Wilcoxon rank-sum (Mann-Whitney-U) test to rank the top 25 genes in each cluster 

### Genes in each cluster are ranked accroding to how different they are relative to all other clusters

In [ ]:
sc.tl.rank_genes_groups(data, 'leiden', method='wilcoxon', corr_method="bonferroni")
sc.pl.rank_genes_groups(data, n_genes=25, sharey=False)

### Using logistic regression to rank the top 25 genes in each cluster 

### Genes in each cluster are ranked accroding to how different they are relative to all other clusters

In [ ]:
sc.tl.rank_genes_groups(data, 'leiden', method='logreg')
sc.pl.rank_genes_groups(data, n_genes=25, sharey=False)

### Selecting the Wilcoxon rank-sum (Mann-Whitney-U) test as the statistical method of choice for ranking genes in each cluster according to how different they are relative to all other clusters

In [ ]:
sc.tl.rank_genes_groups(data, 'leiden', method='wilcoxon', corr_method="bonferroni")

### Using pandas to generate a Dataframe of the genes ranked in each cluster

### Writing the pandas DataFrame to a CSV file

In [ ]:
df = pd.DataFrame(data.uns['rank_genes_groups']['names'])
df.to_csv('Ranked_genes.csv')

### Creating a table to display the top marker gene for each cluster and it's corresponding p-value

In [ ]:
genes = pd.DataFrame(data.uns['rank_genes_groups']['names']).to_numpy()
pvals = pd.DataFrame(data.uns['rank_genes_groups']['pvals']).to_numpy()

In [ ]:
unique = np.unique(data.obs.leiden.values.astype(int)).astype(str)
markers_gene = pd.DataFrame(index=unique, columns=["gene_name", "p_value"])

In [ ]:
for un, u in enumerate(unique):
    g = genes[:,un]
    p = pvals[:,un]
    markers_gene.loc[u]["gene_name"]  = g.tolist()
    markers_gene.loc[u]["p_value"] = p.tolist()

In [ ]:
markers_gene = markers_gene.apply(pd.Series.explode).reset_index()
markers_gene = markers_gene.rename(columns={"index":'leiden'})

In [ ]:
markers_gene.drop_duplicates(["leiden"]) 

### Set up for generating violin plots

In [ ]:
def vplot_de(x, unique, specific_gene, specific_cluster, ax):
    unique = unique.astype(str)
    labels = unique
    lidx = np.arange(1, len(labels)+1)  # the label locations
    midx = np.where(unique==specific_cluster)[0][0]
    
    
    parts = ax.violinplot(x, showmedians=False, showextrema=False)
    for pcidx, pc in enumerate(parts['bodies']):
        pc.set_facecolor('grey')
        pc.set_edgecolor('black')
        pc.set_alpha(1)
        if pcidx == midx:
            pc.set_facecolor('#D43F3A')
            
    mean = [np.mean(i) for i in x]
    ax.scatter(lidx, mean, marker='o', color='white', s=30, zorder=3)
    
    ax.set_ylabel("$log(CPM + 1)$".format(specific_gene))
    ax.set_xticks(lidx)
    ax.set_xticklabels(labels, rotation=0, ha="center")
    ax.set_title("{} gene in cluster {}".format(specific_gene, specific_cluster))
    
    return ax

In [ ]:
specific_cluster = markers_gene.drop_duplicates(["leiden"])["leiden"].values
specific_gene = markers_gene.drop_duplicates(["leiden"])["gene_name"].values

In [ ]:
unique

### Plotting log(CPM+1) distributions for top marker genes in each cluster

In [ ]:
length = len(specific_cluster)*5
height = 3

fig, ax = plt.subplots(figsize=(20,10), ncols = 3, nrows=4)

axs = trim_axs(ax, len(specific_cluster))

for sn, (spec_c, spec_g) in enumerate(zip(specific_cluster, specific_gene)):
    x = []
    for c in unique:
        x.append(nd(data[data.obs.leiden==str(c)][:,data.var.gene_name==spec_g].layers["log1p"].todense()).tolist())
        
    vplot_de(x, unique, spec_g, spec_c, ax=axs[sn])
    
    
fig.text(0.5, 0, 'Leiden cluster', ha='center', va='center', fontsize=30)
fig.text(0, 0.5, '$log(CPM +1)$', ha='center', va='center', rotation='vertical', fontsize=30)
plt.tight_layout()

plt.show()

### Visualising the expression pattrerns of opsin genes 

In [ ]:
gene = "RHO" # Expression pattern of "RHO", to look at the expression pattern of another opsin gene change "RHO" with the gene symbol of another opsin gene, for example "OPN1LW"

fig, ax = plt.subplots(figsize=(7,7))

x = data.obsm["X_nca_tsne"][:,0]
y = data.obsm["X_nca_tsne"][:,1]
c = nd(data.layers["log1p"].todense()[:,data.var.gene_name==gene])

ax.scatter(x, y, c = c, cmap='Reds', alpha=0.5)

ax.set_axis_off()
ax.set_title("{} expression".format(gene))
plt.tight_layout()
plt.show()

### Annotating the t-SNE (using NCA projection) plot with the top marker gene for each cluster

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

c = np.unique(data.obs["leiden"].values.astype(int)).astype(str)
cmap = cm.get_cmap("tab20")

for idx, (cluster, gene) in enumerate(zip(c, specific_gene)):
    XX = data[data.obs.leiden == cluster,:].obsm["X_nca_tsne"]
    
    x = XX[:,0]
    y = XX[:,1]
    ax.scatter(x, y, color = cmap(idx), label=cluster)
    ax.annotate(gene, 
             (np.mean(x), np.mean(y)),
             horizontalalignment='center',
             verticalalignment='center',
             size=15, weight='bold',
             color="white",
               backgroundcolor=cmap(idx)) 
    

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_axis_off()
plt.show()

### Generating a plot displaying CPM distribution for each top marker gene in each cluster 

In [ ]:
top_idx = [np.where(i == data.var.gene_name.values)[0][0] for i in specific_gene]

In [ ]:
mat = data.layers["norm"].todense()
obs = data.obs
var = data.var

In [ ]:
fig, axs = plt.subplots(figsize=(20,20), nrows=len(unique))
fig.subplots_adjust(wspace=0, hspace=0)


labels = specific_gene
lidx = np.arange(0, len(top_idx), 1)
means = []

for cidx, (c, ax) in enumerate(zip(unique, axs)):
    tmp_mat = mat[obs.leiden==str(c),:]
    
    x = tmp_mat[:,top_idx]
    means.append(nd(np.median(x,axis=0)))

    v = ax.violinplot(x.T.tolist(), showmedians=False, showextrema=False, positions=lidx)

    for pcidx, pc in enumerate(v['bodies']):
        pc.set_edgecolor('black')
        pc.set_alpha(1)
        pc.set_facecolor(cm.tab20c(cidx))
        

    means = [np.mean(i) for i in x.T]
    ax.scatter(lidx, means, marker='o', color='white', s=30, zorder=3)
 
    if cidx==0:
        ax_top = ax.twiny()
        ax_top.set_xlim(ax.get_xlim())
        ax_top.set_xticks(lidx)
        ax_top.set_xticklabels(labels, rotation=90, ha="center")
        ax_top.spines["top"].set_visible(True)
        ax_top.spines["left"].set_visible(False)
        ax_top.spines["bottom"].set_visible(False)
    if cidx == len(unique)-1:
        ax_bot = ax.twiny()
        ax_bot.set_xticks([])
        ax_bot.set_xticklabels([])
        ax_bot.spines["top"].set_visible(False)
        ax_bot.spines["left"].set_visible(False)
        ax_bot.spines["bottom"].set_visible(True)

    ax.set_xticklabels("")
    ax.yaxis.tick_right()
    ax.set_ylabel("{} [{:,}]".format(c, x.shape[0]), color="white",rotation="horizontal", ha="right",bbox=dict(boxstyle="square",ec="black",fc=cm.tab20c(cidx)))
    

    
    lim = nd(x.mean(axis=0))[cidx]*4
    
    ax.set_ylim(-lim*0.1, lim)
    
    ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
    
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)

        
    
    

plt.tight_layout(pad=0, w_pad=0, h_pad=0)
fig.text(1, 0.5, 'CPM ', ha='center', va='center', rotation=270, fontsize=30)
plt.show()

### Expression of marker genes for rod and cone photoreceptor cells

### Code in the following two cells is commented out
### Uncomment the code for marker genes for rod photoreceptor cells in the next cell and run the following cells as usual (don't uncomment code for marker genes for cone photoreceptor cells)
### Repeat this process,but now for the code for marker genes for cone photoreceptor cells (comment code for marker genes for rod photoreceptor cells)

#### Marker genes for rod photoreceptor cells

In [ ]:
# markers = ["RHO","NRL","GNAT1","ROM1","CNGA1","SAG","GNGT1"]
# features = data.var.gene_name.values

#### Marker genes for cone photoreceptor cells

In [ ]:
# markers = ["ARR3","GNAT2","GNGT2","GRK7","PDE6C","PDE6H","OPN1LW","RXRG","THRB"]
# features = data.var.gene_name.values

In [ ]:
midx = [np.where(i==features)[0][0] for i in markers]

In [ ]:
assignments = data.obs.leiden.values

In [ ]:
# for each cluster for each gene get two things
# 1 percent of cells in the cluster expressing that gene
# 2 average expression of that gene (for cells that are expressing it)


per = np.zeros((len(unique), len(markers)))
avg = np.zeros((len(unique), len(markers)))

mtx = data.layers["log1p"]#.todense()


for cn, c in enumerate(unique):
    tmp_mtx = mtx[assignments==c]
    sub_mtx = tmp_mtx[:,midx]
    
    avg[cn] = nd(sub_mtx.mean(axis=0))
    per[cn] = (sub_mtx>0).sum(axis=0)/sub_mtx.shape[0]

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
xidx = np.arange(len(markers))
yidx = np.arange(len(unique))

xlabels = markers
ylabels = unique

X, Y = np.meshgrid(xidx, yidx)


for dn, d in enumerate(per):
    a = ax.scatter(X[dn],Y[dn], s=d*500+10, c = avg[dn], cmap="OrRd")

ax.set_xticks(xidx)
ax.set_yticks(yidx)

ax.set_xticklabels(xlabels, rotation=90, ha="center")
ax.set_yticklabels(ylabels)

ax.set_xlabel("Gene")
ax.set_ylabel("Cluster")

ax.figure.colorbar(a, ax=ax, label="$log(CPM+1)$")

handles =  [Line2D([0], [0], marker='o', color='w', label='  0%',markerfacecolor='black', markersize=7),
            Line2D([0], [0], marker='o', color='w', label=' 25%',markerfacecolor='black', markersize=10),
            Line2D([0], [0], marker='o', color='w', label=' 50%',markerfacecolor='black', markersize=12),
            Line2D([0], [0], marker='o', color='w', label=' 75%',markerfacecolor='black', markersize=13.5),
            Line2D([0], [0], marker='o', color='w', label='100%',markerfacecolor='black', markersize=17)]
ax.legend(handles=handles, loc="center left", bbox_to_anchor=(1.15,0.5), title="% of cells")

plt.show()